<a href="https://colab.research.google.com/github/hangyeolS2/HAI-DialectTranslator/blob/main/multi_label_classification/train/train_classifier_gyeol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 입력된 발화의 지역 방언을 구분하는 모델 학습시키기

- AI Hub에서 다운로드 받아 생성한 데이터셋을 이용해 입력된 텍스트가 표준어 발화인지 아니면 특정 지역의 방언인지 분류할 수 있는 모델을 학습시켜 봅시다.
- 먼저 필요한 라이브러리를 설치 및 import해 줍니다.

In [1]:
!pip install transformers

import os
import random
import easydict
import requests
import torch
import numpy as np
import pandas as pd

from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


- 다음은 학습 과정에서 데이터의 전처리와 배치 단위 입력을 수월하게 처리해줄 수 있게 하는 DataLoader를 이용하여 모델 학습을 위한 데이터를 전처리하는 함수입니다.
- generate_data_loader를 호출하면 입력된 파일 경로에서 파일을 읽어와 적절한 토크나이징을 진행하고 args에 정의되어 있는 크기만큼 배치 단위로 데이터를 제공할 수 있는 iteratable한 DataLoader 객체를 반환하게 됩니다.

In [2]:
def generate_data_loader(file_path, tokenizer, args):
    def get_input_ids(data):
        document_bert = ["[CLS] " + str(s) + " [SEP]" for s in data]
        tokenized_texts = [tokenizer.tokenize(s) for s in tqdm(document_bert, "Tokenizing")]
        input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(tokenized_texts, "Converting tokens to ids")]
        print("Padding sequences...")
        input_ids = pad_sequences(input_ids, maxlen=args.maxlen, dtype='long', truncating='post', padding='post')
        return input_ids

    def get_attention_masks(input_ids):
        attention_masks = []
        for seq in tqdm(input_ids, "Generating attention masks"):
            seq_mask = [float(i > 0) for i in seq]
            attention_masks.append(seq_mask)
        return attention_masks

    def get_data_loader(inputs, masks, labels, batch_size=args.batch):
        data = TensorDataset(torch.tensor(inputs), torch.tensor(masks), torch.tensor(labels))
        #sampler = RandomSampler(data)
        sampler = SequentialSampler(data)
        data_loader = DataLoader(data, sampler=sampler, batch_size=batch_size)
        return data_loader

    data_df = pd.read_csv(file_path)
    input_ids = get_input_ids(data_df['contents'].values)
    attention_masks = get_attention_masks(input_ids)
    data_loader = get_data_loader(input_ids, attention_masks, data_df['label'].values if args.mode=='train' else [-1]*len(data_df))

    return data_loader

- 아래 함수는 모델을 학습/추론하는 과정에서 필요한 보조 함수들입니다.
- save는 torch 라이브러리의 state_dict를 저장하는 기능을 이용해 모델의 가중치만 주어진 경로에 저장하는 함수입니다.
- flat_accuracy는 모델이 예측한 결과값과 정답 라벨을 비교하여 얼마나 정확하게 맞혔는지 정확도를 구해주는 함수입니다.

In [3]:
def save(model, dir_name):
    os.makedirs(dir_name, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(dir_name, 'model.pth'))

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

- predict는 학습된 모델을 평가하기 위한 함수입니다. 데이터 입력을 DataLoader 형식으로 받아 모델이 예측한 값을 받아온 뒤 flat_accuracy를 호출하여 정답 라벨과 비교한 정확도를 계산합니다.
- 모델의 추론 과정(Validation 또는 Test 과정)에서 back propagation은 일어나지 않기 때문에, 계산 속도를 높이기 위해 torch.no_grad()를 실행하여 모델에 데이터를 입력해도 gradient가 따로 계산되어 저장되지 않도록 했습니다.

In [4]:
def predict(model, args, data_loader):
    print('start predict')
    model.eval()

    eval_accuracy = []
    logits = []
    
    for step, batch in tqdm(enumerate(data_loader)):
        batch = tuple(t.to(args.device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids,
                            #token_type_ids=None,
                            attention_mask=b_input_mask)
        logit = outputs[0]

        logit = logit.detach().cpu().numpy()
        label = b_labels.cpu().numpy()

        logits.append(logit)

        accuracy = flat_accuracy(logit, label)
        eval_accuracy.append(accuracy)

    logits = np.vstack(logits)
    predict_labels = np.argmax(logits, axis=1)
    return predict_labels, np.mean(eval_accuracy)

- 이 노트북에서 가장 중요한 부분인 train은 모델을 학습시키기 위한 함수입니다. Train data와 Valid data를 각각 DataLoader 형태로 입력받아 학습과 검증 과정을 거치게 됩니다.
- 개선된 optimization 알고리즘인 AdamW와 learning rate를 선형적으로 감소시키는 linear scheduler를 이용하여 학습을 진행합니다.
- 한 epoch가 종료되면 valid_loader를 이용해 predict를 호출하여 validation accuracy를 계산합니다.
- 대부분의 PyTorch를 활용한 모델 학습 과정은 이 함수와 비슷한 과정을 거쳐 진행되니 패턴에 익숙해지면 좋습니다.

In [5]:
def train(model, args, train_loader, valid_loader):
    optimizer = AdamW(model.parameters(),
                      lr=args.lr,
                      eps=args.eps
                      )
    total_steps = len(train_loader) * args.epochs

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)

    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    print('start training')
    for epoch in range(args.epochs):
        model.train()
        train_loss = []
        for step, batch in tqdm(enumerate(train_loader), f"training epoch {epoch}", total=len(train_loader)):
            model.zero_grad()
            batch = tuple(t.to(args.device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            outputs = model(b_input_ids,
                            #token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs[0]
            train_loss.append(loss.item())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = np.mean(train_loss)
        _, avg_val_accuracy = predict(model, args, valid_loader)
        print("Epoch {0},  Average training loss: {1:.4f} , Validation accuracy : {2:.4f}"\
              .format(epoch, avg_train_loss, avg_val_accuracy))

        save(model, "./saved_checkpoints/" + str(epoch))
    return model

- 필요한 함수를 정의 완료했으니 학습을 본격적으로 진행해 봅시다.
- args에는 학습 과정에서 지정해야 할 각종 하이퍼파라미터(배치 사이즈, learning rate 등등)와 데이터 파일 경로 등을 입력해둬 코드 실행 과정에서 사용할 수 있도록 합니다.

In [6]:
args = args = easydict.EasyDict({
  "train_path" : "./data/train_data.csv",
  "valid_path" : "./data/valid_data.csv",
  "device" : 'cpu',
  "mode" : "train",
  "batch" : 128,
  "maxlen" : 128,
  "lr" : 2e-5,
  "eps" : 1e-8,
  "epochs" : 2,
  "model_ckpt" : "monologg/koelectra-base-v3-discriminator",
})

if torch.cuda.is_available():
    args.device = 'cuda'

- 전처리가 완료된 데이터는 우리 프로젝트의 repo에 업로드되어 있어, colab에서 이 노트북을 실행시키더라도 바로 다운로드받아 사용할 수 있습니다.
- AI Hub에 업로드되어 있는 경상도 방언 데이터셋을 어떻게 전처리했는지 코드가 궁금한 사람은 우리 프로젝트 repo 안의 data_preprocessing 디렉토리를 참고하면 되겠습니다.

In [7]:
# download data
os.makedirs('./data', exist_ok=True)

train_url = "https://github.com/GirinMan/HAI-DialectTranslator/raw/main/multi_label_classification/datasets/train_data.csv"
valid_url = "https://github.com/GirinMan/HAI-DialectTranslator/raw/main/multi_label_classification/datasets/valid_data.csv"

train_response = requests.get(train_url, allow_redirects=True)
valid_response = requests.get(valid_url, allow_redirects=True)

open(args.train_path, 'wb').write(train_response.content)
open(args.valid_path, 'wb').write(valid_response.content)    

29160658

- Train/Valid 데이터는 각각 content, label 두 개의 column을 가지는 csv 파일 형태로 이루어져 있습니다. conent는 평가 대상 발화 텍스트가, label은 방언 여부를 나타내는 정수 라벨입니다(표준어일 경우 0, 경상도 방언일 경우 1, 제주도 방언일 경우 2, 전라도 방언일 경우 3).

In [8]:
train_data_df = pd.read_csv(args.train_path)
print(train_data_df.head())
print(train_data_df.tail())

                                            contents  label
0                    머 못 먹 그리고 모든걸 다 좋아하는 데 즐기진 않는다.      0
1                        근데 먹이가 있어야지 얘네가 먹고 자랄꺼 아니야.      0
2                 먹이가 될수 없데 근데 머가 얘들의 먹이는 무엇이냐면 식이섬유      0
3               그러니깐 인제 야채 과일 머 식이섬유 머 자기는 그런거 좋아하니깐      0
4  아니 어쩔수 없이 먹는거 빼고 음식점 선 음식점 선택하기도 그렇고 같이 그 머 식사...      0
                                             contents  label
1942469                              저도 예측하기가 쫌 힘드네요.      3
1942470                            그껴 보면 좋을 것 같은데 @웃음      0
1942471   나이가 먹으면 어른이 되 가지고 좀 점잔 해 지고 그래야 되는데 그렇지는 않네      0
1942472                            그껴 보면 좋을 것 같은디 @웃음      3
1942473   나이가 먹으면 어른이 되 가지고 좀 점잔 해 지고 그래야 되는디 그렇지는 않네      3


- 이제 학습을 위한 모델을 준비해 보겠습니다.
- args에 정의되어 있는 모델의 체크포인트를 이용해 Huggingface hub로부터 sequence classification을 위한 모델과 토크나이저를 불러온 뒤, 모델을 GPU 메모리로 옮깁니다.
- 분류해야 하는 class의 종류가 4가지(표준어, 경상도, 제주도, 전라도)이기 때문에, num_labels를 4로 지정해 주었습니다.

In [9]:
# load model&tokenizer
model = AutoModelForSequenceClassification.from_pretrained(args.model_ckpt, num_labels=4)
model.to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_ckpt)

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

- 미리 정의된 generate_data_loader 함수를 이용해 train/valid 데이터에 대한 DataLoader를 생성합니다.

In [10]:
train_dataloader = generate_data_loader(args.train_path, tokenizer, args)
validation_dataloader = generate_data_loader(args.valid_path, tokenizer, args)

Converting tokens to ids: 100%|██████████| 1942474/1942474 [00:23<00:00, 83477.15it/s] 


Padding sequences...


Converting tokens to ids: 100%|██████████| 440166/440166 [00:04<00:00, 103572.95it/s]


Padding sequences...


Generating attention masks: 100%|██████████| 440166/440166 [00:27<00:00, 16219.72it/s]


- 모델과 하이퍼파라미터 그리고 데이터가 준비되었으니 학습을 진행시켜 봅시다.
- 각 epoch가 끝날 때 마다 모델의 가중치를 저장하고 validation 결과를 출력합니다. 이를 바탕으로 최적의 결과를 가지는 모델을 선택할 수 있습니다.

In [11]:
model = train(model, args, train_dataloader, validation_dataloader)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


start training


training epoch 0:   0%|          | 1/15176 [00:04<18:01:40,  4.28s/it]


RuntimeError: ignored

In [ ]:
input_txt = "안녕하세요"

test = torch.tensor([tokenizer.encode(input_txt)]).to(args.device)

with torch.no_grad():
    preds = model(test).logits.cpu()

result = np.argmax(preds, axis=1).item()

region = ["경상도", "제주도", "전라도"]

print('입력된 문장 "' + input_txt + '"은/는 ', end='')
if result:
    print(region[result-1], "방언입니다.")
else:
    print("표준어 발화입니다.")